pip install pydantic

In [16]:
pip install pytest

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install geopy


   ---------------------------------------- 0.0/125.4 kB ? eta -:--:--
   --- ------------------------------------ 10.2/125.4 kB ? eta -:--:--
   ---------------------------------------  122.9/125.4 kB 2.4 MB/s eta 0:00:01
   ---------------------------------------- 125.4/125.4 kB 1.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/40.3 kB ? eta -:--:--
   ---------------------------------------- 40.3/40.3 kB 2.0 MB/s eta 0:00:00


In [26]:
import pandas as pd
import numpy as np
from math import radians, sin, cos, sqrt, atan2
import unittest
from pydantic import BaseModel
from geopy.distance import geodesic
import pytest

In [25]:
route_data=pd.read_csv('flight_data/routes.csv')
airline_data=pd.read_csv('flight_data/airlines.csv')
airport_data=pd.read_csv('flight_data/airports.csv')
airplane_data=pd.read_csv('flight_data/airplanes.csv')

print(airports)

      index  Airport ID                                         Name  \
0         0           1                               Goroka Airport   
1         1           2                               Madang Airport   
2         2           3                 Mount Hagen Kagamuga Airport   
3         3           4                               Nadzab Airport   
4         4           5  Port Moresby Jacksons International Airport   
...     ...         ...                                          ...   
7693   7693       14106                          Rogachyovo Air Base   
7694   7694       14107                        Ulan-Ude East Airport   
7695   7695       14108                         Krechevitsy Air Base   
7696   7696       14109                  Desierto de Atacama Airport   
7697   7697       14110                           Melitopol Air Base   

              City           Country IATA  ICAO   Latitude   Longitude  \
0           Goroka  Papua New Guinea  GKA  AYGA  -6.081690  1

In [35]:
from geopy.distance import geodesic

class AirportDistanceCalculator:
    def __init__(self, airport_data):
        self.airport_data = airport_data
        self.distances = {}

    def calculate_distance(self, airport_id1, airport_id2):
        # Check if the distance is already calculated and stored
        if (airport_id1, airport_id2) in self.distances:
            return self.distances[(airport_id1, airport_id2)]
        elif (airport_id2, airport_id1) in self.distances:
            return self.distances[(airport_id2, airport_id1)]

        # If not, calculate and store the distance
        coords1 = self.get_coordinates(airport_id1)
        coords2 = self.get_coordinates(airport_id2)
        distance = geodesic(coords1, coords2).kilometers

        self.distances[(airport_id1, airport_id2)] = distance
        return distance

    def get_coordinates(self, airport_id):
        # Implement a method to retrieve coordinates from the dataset
        # This method should use airport_id to find latitude and longitude
        # Example:
        latitude = self.airport_data.loc[self.airport_data['Airport ID'] == airport_id, 'Latitude'].values[0]
        longitude = self.airport_data.loc[self.airport_data['Airport ID'] == airport_id, 'Longitude'].values[0]
        return latitude, longitude

    def haversine_distance(self, lat1, lon1, lat2, lon2):
        # Implement the haversine formula if needed
        pass
    

In [40]:
from math import radians, sin, cos, sqrt, atan2

def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of the Earth in kilometers

    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    # Distance in kilometers
    distance = R * c

    return distance

# Coordinates of airports
lat1, lon1 = -6.081690, 145.391998
lat2, lon2 = -5.207080, 145.789001
lat3, lon3 = -5.826790, 144.296005

# Calculate distances using the Haversine formula
distance_same_airport = haversine(lat1, lon1, lat1, lon1)
distance_diff_airports = haversine(lat1, lon1, lat2, lon2)
distance_diff_continents = haversine(lat1, lon1, lat3, lon3)

print("Expected distances:")
print(f"Same airport: {distance_same_airport:.2f} km")
print(f"Different airports: {distance_diff_airports:.2f} km")
print(f"Different continents: {distance_diff_continents:.2f} km")

Expected distances:
Same airport: 0.00 km
Different airports: 106.71 km
Different continents: 124.48 km


In [39]:
from geopy.distance import geodesic
import unittest
import pandas as pd
from distance_calculator import AirportDistanceCalculator

class TestAirportDistanceCalculator(unittest.TestCase):
    def setUp(self):
        # Sample airport data (using the provided dataset)
        self.airport_data = pd.DataFrame({
            'Airport ID': [1, 2, 3],
            'Latitude': [-6.081690, -5.207080, -5.826790],
            'Longitude': [145.391998, 145.789001, 144.296005]
        })
        self.calculator = AirportDistanceCalculator(self.airport_data)

    def test_distance_between_same_airport(self):
        distance = self.calculator.calculate_distance(1, 1)
        self.assertEqual(distance, 0)

    def test_distance_between_different_airports(self):
        # Replace the coordinates and expected_distance with actual values
        lat1, lon1 = self.airport_data.loc[self.airport_data['Airport ID'] == 1, ['Latitude', 'Longitude']].values[0]
        lat2, lon2 = self.airport_data.loc[self.airport_data['Airport ID'] == 2, ['Latitude', 'Longitude']].values[0]
        distance = geodesic((lat1, lon1), (lat2, lon2)).kilometers
        expected_distance = 106.24895585297145 # Replace with the actual distance
        self.assertAlmostEqual(distance, expected_distance, places=2)

    def test_distance_between_airports_in_different_continents(self):
        # Replace the coordinates and expected_distance with actual values
        lat1, lon1 = self.airport_data.loc[self.airport_data['Airport ID'] == 1, ['Latitude', 'Longitude']].values[0]
        lat3, lon3 = self.airport_data.loc[self.airport_data['Airport ID'] == 3, ['Latitude', 'Longitude']].values[0]
        distance = geodesic((lat1, lon1), (lat3, lon3)).kilometers
        expected_distance = 124.5823203460621 # Replace with the actual distance
        self.assertAlmostEqual(distance, expected_distance, places=2)

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

...
----------------------------------------------------------------------
Ran 3 tests in 0.010s

OK
